In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import itertools
import os

def create_dataset(file_paths,stats,method=None):
    if len(file_paths) > 1:
        df1 = pd.read_csv(file_paths[0],sep=',',index_col=None)#,nrows=10000)
        df2 = pd.read_csv(file_paths[1],sep=',',index_col=None)#,nrows=10000)
        df3 = pd.read_csv(file_paths[2],sep=',',index_col=None)#,nrows=10000)
        df = pd.concat([df1,df2,df3],axis=0)                   # Useful for debugging
    else:
        df = pd.read_csv(file_paths[0],sep=',',index_col=None)
    
    if method == 'Simulation':
        top_row = df[(df.y > 248) & (df.x > 348)]
        middle = df[(df.y < 248) & (df.y > 48)]
        bottom_row = df[(df.y < 48) & (df.x > 548)]
        df = pd.concat([top_row,middle,bottom_row])
        
    df = df.to_numpy()
    print(len(df))
    hits = df[:,:3]
    conds = df[:,3:6]
    metadata = df[:,6:]
    return hits,conds,metadata

In [2]:
if not os.path.exists("Figures"):
    os.makedirs("Figures")

In [3]:
config_sim = json.load(open("config/inv_mass_config.json"))
config_data = json.load(open("config/real_config.json"))

# Start with Kaons

In [4]:
file_paths_sim = [config_sim['dataset']['training']['unsmeared']['kaon_data_path'],
                      config_sim['dataset']['validation']['kaon_data_path'],
                      config_sim['dataset']['testing']['gen']['kaon_data_path']]



file_paths_data = [config_data['dataset']['training']['unsmeared']['kaon_data_path'],
                      config_data['dataset']['validation']['kaon_data_path'],
                      config_data['dataset']['testing']['gen']['kaon_data_path']]


sim_kaons,sim_kaon_conds,sim_meta_kaons = create_dataset(file_paths_sim,config_sim['stats'],'Simulation')
data_kaons,data_kaon_conds,data_meta_kaons = create_dataset(file_paths_data,config_data['stats'])

15976883
29701825


In [5]:
def make_comparison_plot(simulation,data,p_low,p_high,barID,PID,folder_):
    
    fig,ax = plt.subplots(1,3,figsize=(16,4))
    ax = ax.ravel()
    ld= len(data[:,0])
    ls = len(simulation[:,0])
    if PID == 'Kaon':
        sim_label = r'$\mathcal{K}_{sim.}$ - '+ '{0}'.format(ls)
        data_label = r'$\mathcal{K}$ - '+ '{0}'.format(ld)
    elif PID == 'Pion':
        sim_label = r'$\pi_{sim.}$ - '+ '{0}'.format(ls)
        data_label = r'$\pi$ - '+ '{0}'.format(ld)
        
    bins_x = np.array([0,  3.,   9.,  15.,  21.,  27.,  33.,  39.,  45.,48,50, 53.,  59.,  65.,
        71.,  77.,  83.,  89.,  95., 98,100, 103., 109., 115., 121., 127., 133.,
       139., 145.,148,150, 153., 159., 165., 171., 177., 183., 189., 195.,198,200, 203.,
       209., 215., 221., 227., 233., 239., 245.,248,250, 253., 259., 265., 271.,
       277., 283., 289., 295.,298,300, 303., 309., 315., 321., 327., 333., 339.,
       345.,348,350, 353., 359., 365., 371., 377., 383., 389., 395.,398,400, 403., 409.,
       415., 421., 427., 433., 439., 445.,448,450, 453., 459., 465., 471., 477.,
       483., 489., 495.,498,500, 503., 509., 515., 521., 527., 533., 539., 545.,548,550,
       553., 559., 565., 571., 577., 583., 589., 595.,598,600, 603., 609., 615.,
       621., 627., 633., 639., 645.,648,650, 653., 659., 665., 671., 677., 683.,
       689., 695.,698,700, 703., 709., 715., 721., 727., 733., 739., 745.,748,750, 753.,
       759., 765., 771., 777., 783., 789., 795.,798,800, 803., 809., 815., 821.,
       827., 833., 839., 845.848,850, 853., 859., 865., 871., 877., 883., 889.,
       895.,898,900])
    
    
    bins_y = np.array([0,  3.,   9.,  15.,  21.,  27.,  33.,  39.,  45.,48,50, 53.,  59.,  65.,
        71.,  77.,  83.,  89.,  95.,98,100, 103., 109., 115., 121., 127., 133.,
       139., 145.,148,150, 153., 159., 165., 171., 177., 183., 189., 195.,198,200, 203.,
       209., 215., 221., 227., 233., 239., 245.,248,250, 253., 259., 265., 271.,
       277., 283., 289., 295.,298,300])
    
    time_bins = np.arange(0,200+1,1)
        
    ax[0].hist(simulation[:,0],bins=bins_x,density=True,histtype='step',color='k',linewidth=2,label=sim_label)
    ax[0].hist(data[:,0],bins=bins_x,density=True,histtype='step',color='r',linewidth=2,label=data_label)
    ax[0].legend(loc='best',fontsize=18)
    ax[0].set_xlabel("X (mm)",fontsize=20)
    ax[0].set_ylabel("Density",fontsize=20)
    
    ax[1].hist(simulation[:,1],bins=bins_y,density=True,histtype='step',color='k',linewidth=2,label=sim_label)
    ax[1].hist(data[:,1],bins=bins_y,density=True,histtype='step',color='r',linewidth=2,label=data_label)
    ax[1].legend(loc='best',fontsize=18)
    ax[1].set_xlabel("Y (mm)",fontsize=20)
    #ax[1].set_ylabel("Density",fontsize=20)
    
    ax[2].hist(simulation[:,2],bins=time_bins,density=True,histtype='step',color='k',linewidth=2,label=sim_label)
    ax[2].hist(data[:,2],bins=time_bins,density=True,histtype='step',color='r',linewidth=2,label=data_label)
    ax[2].legend(loc='best',fontsize=18)
    ax[2].set_xlabel("Time (ns)",fontsize=20)
    #ax[2].set_ylabel("Density",fontsize=20)
    
    ax[1].set_title(r"Comparison of Simulated and Actual DIRC Response - Bar {0}, $p \in ({1:.2f},{2:.2f})$".format(barID,p_low,p_high),fontsize=20,pad=20)

    if not os.path.exists(folder_):
        os.makedirs(folder_)
    path_ = os.path.join(folder_,"Comp_p({0:.2f},{1:.2f})_BarID{2}.pdf".format(p_low,p_high,barID))
    plt.savefig(path_,bbox_inches="tight")
    plt.close()
    
    
    

In [6]:
def run(simulation,data,PID=None,folder=None):
    assert PID is not None
    assert folder is not None
    
    delta_p = 0.1 # 100 MeV
    delta_phi = 0.26 # ~ 15 degrees
    mr = np.arange(1.,np.max(sim_kaon_conds[:,0])+delta_p,delta_p)
    #tr = np.arange(np.min(sim_kaon_conds[:,1]),np.max(sim_kaon_conds[:,1])+delta_phi,delta_phi)
    bars = np.arange(0,48+1,1)
    momentum_ranges = []
    for i in range(len(mr) - 1):
        momentum_ranges.append((mr[i],mr[i+1]))
    #theta_ranges = []
    #for i in range(len(tr) - 1):
    #    theta_ranges.append((tr[i],tr[i+1]))
    combinations = list(itertools.product(momentum_ranges,bars))
    
    data_hits,data_conds,data_meta = data
    sim_hits,sim_conds,sim_meta = simulation
    
    for combo in combinations:
        #print(combo)
        p_low = combo[0][0]
        p_high = combo[0][1]
        bar = combo[1]
        
        d_idx = np.where((data_conds[:,0] > p_low) & (data_conds[:,0] < p_high) & (data_meta[:,0] == bar))[0]
        s_idx = np.where((sim_conds[:,0] > p_low) & (sim_conds[:,0] < p_high) & (sim_meta[:,0] == bar))[0]
        if (len(d_idx) == 0) or (len(s_idx) == 0):
            #print('No data at p ({0},{1}) and theta ({2},{3})'.format(p_low,p_high,theta_low,theta_high))
            continue
        #else:
        data_ = data_hits[d_idx]
        simulation_ = sim_hits[s_idx]
        make_comparison_plot(simulation_,data_,p_low,p_high,bar,PID,folder)
        
        
        

In [7]:
run([sim_kaons,sim_kaon_conds,sim_meta_kaons],[data_kaons,data_kaon_conds,data_meta_kaons],'Kaon',"Figures/ComparisonSimData_Kaons")

In [8]:
import glob
from PyPDF2 import PdfWriter

def merge_PDF(out_dir,method):
    pdf_dir = os.path.join(out_dir,'ComparisonSimData_' + str(method))
    pdf_files = sorted(glob.glob(os.path.join(pdf_dir, '*.pdf')))

    pdf_files.sort(key=lambda x: float(x.split('_p(')[-1].split(')')[0].split(',')[0]))
    output_pdf = PdfWriter()

    for pdf_file in pdf_files:
        with open(pdf_file, 'rb') as f:
            pdf_data = f.read()
            output_pdf.append(fileobj=f)

    # Save the combined PDF to a file
    with open(os.path.join(out_dir,'Combined_SimVsData_'+str(method)+'.pdf'), 'wb') as f:
        output_pdf.write(f)

In [9]:
merge_PDF("Figures",'Kaons')

In [10]:
import matplotlib.colors as mcolors

def make_plot(generations,hits,barID,x_high,x_low,method=None,folder=None):
    # Matches physical sensor
    bins_x = np.array([0,  3.,   9.,  15.,  21.,  27.,  33.,  39.,  45.,48,50, 53.,  59.,  65.,
        71.,  77.,  83.,  89.,  95., 98,100, 103., 109., 115., 121., 127., 133.,
       139., 145.,148,150, 153., 159., 165., 171., 177., 183., 189., 195.,198,200, 203.,
       209., 215., 221., 227., 233., 239., 245.,248,250, 253., 259., 265., 271.,
       277., 283., 289., 295.,298,300, 303., 309., 315., 321., 327., 333., 339.,
       345.,348,350, 353., 359., 365., 371., 377., 383., 389., 395.,398,400, 403., 409.,
       415., 421., 427., 433., 439., 445.,448,450, 453., 459., 465., 471., 477.,
       483., 489., 495.,498,500, 503., 509., 515., 521., 527., 533., 539., 545.,548,550,
       553., 559., 565., 571., 577., 583., 589., 595.,598,600, 603., 609., 615.,
       621., 627., 633., 639., 645.,648,650, 653., 659., 665., 671., 677., 683.,
       689., 695.,698,700, 703., 709., 715., 721., 727., 733., 739., 745.,748,750, 753.,
       759., 765., 771., 777., 783., 789., 795.,798,800, 803., 809., 815., 821.,
       827., 833., 839., 845.848,850, 853., 859., 865., 871., 877., 883., 889.,
       895.,898,900])
    
    bins_y = np.array([0,  3.,   9.,  15.,  21.,  27.,  33.,  39.,  45.,48,50, 53.,  59.,  65.,
        71.,  77.,  83.,  89.,  95.,98,100, 103., 109., 115., 121., 127., 133.,
       139., 145.,148,150, 153., 159., 165., 171., 177., 183., 189., 195.,198,200, 203.,
       209., 215., 221., 227., 233., 239., 245.,248,250, 253., 259., 265., 271.,
       277., 283., 289., 295.,298,300])
    
    fig,ax = plt.subplots(4,2,figsize=(18,16))
    ax = ax.ravel()
    # We have a 2mm offset.
    x_true = hits[:,0] - 2
    y_true = hits[:,1] - 2
    t_true = hits[:,2]

    ax[0].hist2d(x_true,y_true,density=True,bins=[bins_x,bins_y],norm=mcolors.LogNorm())
    ax[0].set_xlabel(r'X $(mm)$',fontsize=20)
    ax[0].set_ylabel(r'Y $(mm)$',fontsize=20)
    # Time PDF
    ax[2].hist(t_true,density=True,color='blue',label='Truth',bins=100,range=[0,200])
    ax[2].set_title("Hit Time",fontsize=20)
    ax[2].set_xlabel("Hit Time (ns)",fontsize=20)
    ax[2].set_ylabel("Density",fontsize=20)
    # X PDF
    ax[4].hist(x_true,density=True,color='blue',label='Truth',bins=100,range=[0,895])
    ax[4].set_title("X Distribution",fontsize=20)
    ax[4].set_xlabel("X (mm)",fontsize=20)
    ax[4].set_ylabel("Density",fontsize=20)
    # Y PDF
    ax[6].hist(y_true,density=True,color='blue',label='Truth',bins=100,range=[0,295])
    ax[6].set_title("Y Distribution",fontsize=20)
    ax[6].set_xlabel("Y (mm)",fontsize=20)
    ax[6].set_ylabel("Density",fontsize=20)

    # Also a 2mm offset it seems. Probably needs correction only after so many PMTs
    x = generations[:,0].flatten() - 2
    y = generations[:,1].flatten() - 2
    t = generations[:,2].flatten() 
    ax[1].hist2d(x,y,density=True,bins=[bins_x,bins_y],norm=mcolors.LogNorm())
    ax[1].set_xlabel(r'X $(mm)$',fontsize=20)
    ax[1].set_ylabel(r'Y $(mm)$',fontsize=20)
    # Time PDF
    ax[3].hist(t,density=True,color='blue',label='Truth',bins=100,range=[0,200])
    ax[3].set_title("Simulated Hit Time",fontsize=20)
    ax[3].set_xlabel("Hit Time (ns)",fontsize=20)
    ax[3].set_ylabel("Density",fontsize=20)
    # X PDF
    ax[5].hist(x,density=True,color='blue',label='Truth',bins=100,range=[0,895])
    ax[5].set_title("Simulated X Distribution",fontsize=20)
    ax[5].set_xlabel("X (mm)",fontsize=20)
    ax[5].set_ylabel("Density",fontsize=20)
    # Y PDF
    ax[7].hist(y,density=True,color='blue',label='Truth',bins=100,range=[0,295])
    ax[7].set_title("Simulated Y Distribution",fontsize=20)
    ax[7].set_xlabel("Y (mm)",fontsize=20)
    ax[7].set_ylabel("Density",fontsize=20)

    plt.subplots_adjust(hspace=0.5)
    if method == "Pion":
        path_ = os.path.join(folder,"Pions_BarID{0}_x({1},{2}).pdf".format(barID,x_low,x_high))
        ax[1].set_title(r'Simulated Pions: $x \in ({0},{1})$,BarID: {2}'.format(x_low,x_high,barID),fontsize=20)
        ax[0].set_title(r'Pions: $x \in ({0},{1})$,BarID: {2}'.format(x_low,x_high,barID),fontsize=20)
        plt.savefig(path_,bbox_inches="tight")
        
    elif method == "Kaon":
        path_ = os.path.join(folder,"Kaons_BarID{0}_x({1},{2}).pdf".format(barID,x_low,x_high))
        ax[1].set_title(r'Simulated Kaons: $x \in ({0},{1})$,BarID: {2}'.format(x_low,x_high,barID),fontsize=20)
        ax[0].set_title(r'Kaons: $x \in ({0},{1})$,BarID: {2}'.format(x_low,x_high,barID),fontsize=20)
        plt.savefig(path_,bbox_inches="tight")
    
    plt.close(fig)

In [11]:
def run_xy_plots(sim,sim_meta,data,data_meta,method=None,folder="Figures/XY_Comp_Kaons"):
    assert method is not None
# Control what you want to generate pair wise here:
    xs = [(-30,-20),(-20,-10),(-10,0),(0,10),(10,20),(20,30)]
    bars = [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

    combinations = list(itertools.product(xs,bars))
    print('Generating PDFs for {0} combinations of BarID and x ranges.'.format(len(combinations)))

    for j,combination in enumerate(combinations):
        x_low = combination[0][0]
        x_high = combination[0][1]
        barID = combination[1]
        sim_idx = np.where((sim_meta[:,0] == barID) & (sim_meta[:,1] > x_low) & (sim_meta[:,1] < x_high))[0]
        sim_subset = sim[sim_idx]
        data_idx = np.where((data_meta[:,0] == barID) & (data_meta[:,1] > x_low) & (data_meta[:,1] < x_high))[0]
        data_subset = data[data_idx]
        if (len(sim_subset) == 0) or (len(data_subset) == 0):
            print(" ")
            print('No data at Bar {0}, x ({1},{2})'.format(barID,x_low,x_high))
            print(" ")
            continue
            
        if not os.path.exists(folder):
            os.mkdir(folder)

        make_plot(sim_subset,data_subset,barID,x_high,x_low,method,folder)

In [12]:
run_xy_plots(sim_kaons,sim_meta_kaons,data_kaons,data_meta_kaons,method='Kaon',folder="Figures/XY_Comp_Kaons")

Generating PDFs for 288 combinations of BarID and x ranges.
 
No data at Bar 19, x (-30,-20)
 
 
No data at Bar 20, x (-30,-20)
 
 
No data at Bar 21, x (-30,-20)
 
 
No data at Bar 22, x (-30,-20)
 
 
No data at Bar 23, x (-30,-20)
 
 
No data at Bar 43, x (-30,-20)
 
 
No data at Bar 44, x (-30,-20)
 
 
No data at Bar 46, x (-30,-20)
 
 
No data at Bar 47, x (-30,-20)
 
 
No data at Bar 21, x (-20,-10)
 
 
No data at Bar 22, x (-20,-10)
 
 
No data at Bar 23, x (-20,-10)
 
 
No data at Bar 43, x (-20,-10)
 
 
No data at Bar 44, x (-20,-10)
 
 
No data at Bar 45, x (-20,-10)
 
 
No data at Bar 46, x (-20,-10)
 
 
No data at Bar 47, x (-20,-10)
 
 
No data at Bar 20, x (-10,0)
 
 
No data at Bar 22, x (-10,0)
 
 
No data at Bar 23, x (-10,0)
 
 
No data at Bar 45, x (-10,0)
 
 
No data at Bar 46, x (-10,0)
 
 
No data at Bar 47, x (-10,0)
 
 
No data at Bar 19, x (0,10)
 
 
No data at Bar 20, x (0,10)
 
 
No data at Bar 21, x (0,10)
 
 
No data at Bar 22, x (0,10)
 
 
No data at Bar 23

In [13]:
import glob
from PyPDF2 import PdfWriter

def merge_PDF_XY(out_dir,method):
    pdf_dir = os.path.join(out_dir,'XY_Comp_' + str(method))
    pdf_files = sorted(glob.glob(os.path.join(pdf_dir, '*.pdf')))

    pdf_files.sort(key=lambda x: float(x.split('BarID')[-1].split('_')[0].split(',')[0]))
    output_pdf = PdfWriter()

    for pdf_file in pdf_files:
        with open(pdf_file, 'rb') as f:
            pdf_data = f.read()
            output_pdf.append(fileobj=f)

    # Save the combined PDF to a file
    with open(os.path.join(out_dir,'Combined_XYComp_'+str(method)+'.pdf'), 'wb') as f:
        output_pdf.write(f)

In [14]:
merge_PDF_XY("Figures","Kaons")

# And now Pions

In [15]:
file_paths_sim = [config_sim['dataset']['training']['unsmeared']['pion_data_path'],
                      config_sim['dataset']['validation']['pion_data_path'],
                      config_sim['dataset']['testing']['gen']['pion_data_path']]



file_paths_data = [config_data['dataset']['training']['unsmeared']['pion_data_path'],
                      config_data['dataset']['validation']['pion_data_path'],
                      config_data['dataset']['testing']['gen']['pion_data_path']]


sim_pions,sim_pion_conds,sim_meta_pions = create_dataset(file_paths_sim,config_sim['stats'],method='Simulation')
data_pions,data_pion_conds,data_meta_pions = create_dataset(file_paths_data,config_data['stats'])

59829163
46146947


In [16]:
run([sim_pions,sim_pion_conds,sim_meta_pions],[data_pions,data_pion_conds,data_meta_pions],'Pion',"Figures/ComparisonSimData_Pions")

In [17]:
merge_PDF("Figures",'Pions')

In [18]:
run_xy_plots(sim_pions,sim_meta_pions,data_pions,data_meta_pions,method='Pion',folder="Figures/XY_Comp_Pions")

Generating PDFs for 288 combinations of BarID and x ranges.


In [19]:
merge_PDF_XY("Figures","Pions")

# Isolate a Single PMT

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import itertools
import os

def create_dataset(file_paths,stats):
    if len(file_paths) > 1:
        df1 = pd.read_csv(file_paths[0],sep=',',index_col=None)#,nrows=10000)
        df2 = pd.read_csv(file_paths[1],sep=',',index_col=None)#,nrows=10000)
        df3 = pd.read_csv(file_paths[2],sep=',',index_col=None)#,nrows=10000)
        df = pd.concat([df1,df2,df3],axis=0)                   # Useful for debugging
    else:
        df = pd.read_csv(file_paths[0],sep=',',index_col=None)
    
    df = df[(df.y > 50) & (df.y < 100) & (df.x > 100) & (df.x < 150) & (df.P < 4.0) & (df.P > 3.5)]
        
    df = df.to_numpy()
    print(len(df))
    hits = df[:,:3]
    conds = df[:,3:6]
    metadata = df[:,6:]
    return hits,conds,metadata

In [32]:
file_paths_sim = [config_sim['dataset']['training']['unsmeared']['kaon_data_path'],
                      config_sim['dataset']['validation']['kaon_data_path'],
                      config_sim['dataset']['testing']['gen']['kaon_data_path']]



file_paths_data = [config_data['dataset']['training']['unsmeared']['kaon_data_path'],
                      config_data['dataset']['validation']['kaon_data_path'],
                      config_data['dataset']['testing']['gen']['kaon_data_path']]


sim_kaons,sim_kaon_conds,sim_meta_kaons = create_dataset(file_paths_sim,config_sim['stats'])
data_kaons,data_kaon_conds,data_meta_kaons = create_dataset(file_paths_data,config_data['stats'])

124256
135255


In [38]:
run_xy_plots(sim_kaons,sim_meta_kaons,data_kaons,data_meta_kaons,method='Kaon',folder="Figures/SinglePMT_Kaons")

Generating PDFs for 288 combinations of BarID and x ranges.
 
No data at Bar 17, x (-30,-20)
 
 
No data at Bar 18, x (-30,-20)
 
 
No data at Bar 19, x (-30,-20)
 
 
No data at Bar 20, x (-30,-20)
 
 
No data at Bar 21, x (-30,-20)
 
 
No data at Bar 22, x (-30,-20)
 
 
No data at Bar 23, x (-30,-20)
 
 
No data at Bar 39, x (-30,-20)
 
 
No data at Bar 41, x (-30,-20)
 
 
No data at Bar 42, x (-30,-20)
 
 
No data at Bar 43, x (-30,-20)
 
 
No data at Bar 44, x (-30,-20)
 
 
No data at Bar 45, x (-30,-20)
 
 
No data at Bar 46, x (-30,-20)
 
 
No data at Bar 47, x (-30,-20)
 
 
No data at Bar 15, x (-20,-10)
 
 
No data at Bar 19, x (-20,-10)
 
 
No data at Bar 20, x (-20,-10)
 
 
No data at Bar 21, x (-20,-10)
 
 
No data at Bar 22, x (-20,-10)
 
 
No data at Bar 23, x (-20,-10)
 
 
No data at Bar 39, x (-20,-10)
 
 
No data at Bar 41, x (-20,-10)
 
 
No data at Bar 42, x (-20,-10)
 
 
No data at Bar 43, x (-20,-10)
 
 
No data at Bar 44, x (-20,-10)
 
 
No data at Bar 45, x (-20,-1

C:\Users\James\.conda\envs\ptorch\lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


 
No data at Bar 40, x (10,20)
 
 
No data at Bar 41, x (10,20)
 
 
No data at Bar 42, x (10,20)
 
 
No data at Bar 43, x (10,20)
 
 
No data at Bar 44, x (10,20)
 
 
No data at Bar 45, x (10,20)
 
 
No data at Bar 46, x (10,20)
 
 
No data at Bar 47, x (10,20)
 
 
No data at Bar 17, x (20,30)
 
 
No data at Bar 18, x (20,30)
 
 
No data at Bar 19, x (20,30)
 
 
No data at Bar 20, x (20,30)
 
 
No data at Bar 21, x (20,30)
 
 
No data at Bar 22, x (20,30)
 
 
No data at Bar 23, x (20,30)
 
 
No data at Bar 39, x (20,30)
 
 
No data at Bar 42, x (20,30)
 
 
No data at Bar 43, x (20,30)
 
 
No data at Bar 44, x (20,30)
 
 
No data at Bar 45, x (20,30)
 
 
No data at Bar 46, x (20,30)
 
 
No data at Bar 47, x (20,30)
 


In [39]:
import glob
from PyPDF2 import PdfWriter

def merge_PDF_SinglePMT(out_dir,method):
    pdf_dir = os.path.join(out_dir,'SinglePMT_' + str(method))
    pdf_files = sorted(glob.glob(os.path.join(pdf_dir, '*.pdf')))

    pdf_files.sort(key=lambda x: float(x.split('BarID')[-1].split('_')[0].split(',')[0]))
    output_pdf = PdfWriter()

    for pdf_file in pdf_files:
        with open(pdf_file, 'rb') as f:
            pdf_data = f.read()
            output_pdf.append(fileobj=f)

    # Save the combined PDF to a file
    with open(os.path.join(out_dir,'Combined_SinglePMT_'+str(method)+'.pdf'), 'wb') as f:
        output_pdf.write(f)

In [40]:
merge_PDF_SinglePMT("Figures","Kaons")

In [41]:
file_paths_sim = [config_sim['dataset']['training']['unsmeared']['pion_data_path'],
                      config_sim['dataset']['validation']['pion_data_path'],
                      config_sim['dataset']['testing']['gen']['pion_data_path']]



file_paths_data = [config_data['dataset']['training']['unsmeared']['pion_data_path'],
                      config_data['dataset']['validation']['pion_data_path'],
                      config_data['dataset']['testing']['gen']['pion_data_path']]


sim_pions,sim_pion_conds,sim_meta_pions = create_dataset(file_paths_sim,config_sim['stats'])
data_pions,data_pion_conds,data_meta_pions = create_dataset(file_paths_data,config_data['stats'])

140625
100920


In [42]:
run_xy_plots(sim_kaons,sim_meta_kaons,data_kaons,data_meta_kaons,method='Pion',folder="Figures/SinglePMT_Pions")

Generating PDFs for 288 combinations of BarID and x ranges.
 
No data at Bar 17, x (-30,-20)
 
 
No data at Bar 18, x (-30,-20)
 
 
No data at Bar 19, x (-30,-20)
 
 
No data at Bar 20, x (-30,-20)
 
 
No data at Bar 21, x (-30,-20)
 
 
No data at Bar 22, x (-30,-20)
 
 
No data at Bar 23, x (-30,-20)
 
 
No data at Bar 39, x (-30,-20)
 
 
No data at Bar 41, x (-30,-20)
 
 
No data at Bar 42, x (-30,-20)
 
 
No data at Bar 43, x (-30,-20)
 
 
No data at Bar 44, x (-30,-20)
 
 
No data at Bar 45, x (-30,-20)
 
 
No data at Bar 46, x (-30,-20)
 
 
No data at Bar 47, x (-30,-20)
 
 
No data at Bar 15, x (-20,-10)
 
 
No data at Bar 19, x (-20,-10)
 
 
No data at Bar 20, x (-20,-10)
 
 
No data at Bar 21, x (-20,-10)
 
 
No data at Bar 22, x (-20,-10)
 
 
No data at Bar 23, x (-20,-10)
 
 
No data at Bar 39, x (-20,-10)
 
 
No data at Bar 41, x (-20,-10)
 
 
No data at Bar 42, x (-20,-10)
 
 
No data at Bar 43, x (-20,-10)
 
 
No data at Bar 44, x (-20,-10)
 
 
No data at Bar 45, x (-20,-1

C:\Users\James\.conda\envs\ptorch\lib\site-packages\numpy\lib\histograms.py:883: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


 
No data at Bar 40, x (10,20)
 
 
No data at Bar 41, x (10,20)
 
 
No data at Bar 42, x (10,20)
 
 
No data at Bar 43, x (10,20)
 
 
No data at Bar 44, x (10,20)
 
 
No data at Bar 45, x (10,20)
 
 
No data at Bar 46, x (10,20)
 
 
No data at Bar 47, x (10,20)
 
 
No data at Bar 17, x (20,30)
 
 
No data at Bar 18, x (20,30)
 
 
No data at Bar 19, x (20,30)
 
 
No data at Bar 20, x (20,30)
 
 
No data at Bar 21, x (20,30)
 
 
No data at Bar 22, x (20,30)
 
 
No data at Bar 23, x (20,30)
 
 
No data at Bar 39, x (20,30)
 
 
No data at Bar 42, x (20,30)
 
 
No data at Bar 43, x (20,30)
 
 
No data at Bar 44, x (20,30)
 
 
No data at Bar 45, x (20,30)
 
 
No data at Bar 46, x (20,30)
 
 
No data at Bar 47, x (20,30)
 


In [43]:
merge_PDF_SinglePMT("Figures","Pions")

In [47]:
data_pion_conds.max(0)

array([  3.999975,  11.537402, 173.798859])